### Gold Medallion/ Layer Processing for Global Earthquake Events

In [1]:
# Import all the necessary libraries for the Gold Medallion data handling

from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType

# ensure the below library is installed on your fabric environment
import reverse_geocoder as rg   # To add location/ countries geocode using the longitude and latitude based on an external libraries

StatementMeta(, e2343822-04e0-4b3c-8cfc-e8502b405780, 3, Finished, Available, Finished)

ModuleNotFoundError: No module named 'reverse_geocoder'

In [2]:
# Read the table

df = spark.read.table("earthquake_events_silver")#.filter(col('time') > start_date)

StatementMeta(, 6a554327-fa65-4dd1-8094-5f76ee6a647e, 4, Finished, Available, Finished)

In [3]:
# Display the table
display(df)

StatementMeta(, 6a554327-fa65-4dd1-8094-5f76ee6a647e, 5, Finished, Available, Finished)

In [2]:
# To check whether the imported reverse_geocoder environment is working
test_coordinates = (36.496,-113.9852) # id-15

rg.search(test_coordinates)[0].get('cc')  # To get the ISO country code which is "US"

StatementMeta(, e2343822-04e0-4b3c-8cfc-e8502b405780, 4, Finished, Available, Finished)

NameError: name 'rg' is not defined

In [5]:
#  Create a function to retrieve the country code for each given latitude and longitude.
# lat (float or str): Latitude of the location.
# lon (float or str): Longitude of the location.
# Returns str: Country code of the location, retrieved using the reverse geocoding API.

def iso_country_code(lat, lon):
    coordinates = (float(lat), float(lon))
    return rg.search(coordinates)[0].get('cc')

StatementMeta(, 6a554327-fa65-4dd1-8094-5f76ee6a647e, 7, Finished, Available, Finished)

In [6]:
# register the udfs so they we can it on spark dataframes
iso_country_code_udf = udf(iso_country_code, StringType())

StatementMeta(, 6a554327-fa65-4dd1-8094-5f76ee6a647e, 8, Finished, Available, Finished)

In [3]:
# adding country_code and city attributes
df_with_location = \
                df.\
                    withColumn("country_code", iso_country_code_udf(col("latitude"), col("longitude")))

StatementMeta(, e2343822-04e0-4b3c-8cfc-e8502b405780, 5, Finished, Available, Finished)

NameError: name 'df' is not defined

In [8]:
df_with_location.show()

StatementMeta(, 6a554327-fa65-4dd1-8094-5f76ee6a647e, 10, Finished, Available, Finished)

+------------+-----------------+----------------+-----------------+--------------------+--------------------+---+-----+-------+--------------------+--------------------+
|          id|        longitude|        latitude|        elevation|               title|   place_description|sig|  mag|magType|                time|             updated|
+------------+-----------------+----------------+-----------------+--------------------+--------------------+---+-----+-------+--------------------+--------------------+
|ak0255aktith|         -150.778|         60.6953|             44.9|M 1.3 - 17 km N o...|17 km N of Sterli...| 26|  1.3|     ml|2025-04-25 23:54:...|2025-04-25 23:57:...|
|  av93040049|-155.340833333333|          58.181|             1.66|M -0.2 - 86 km NW...|86 km NW of Karlu...|  0|-0.17|     ml|2025-04-25 23:49:...|2025-04-27 04:05:...|
|  nn00896815|          -118.41|         38.4903|              1.6|M 1.7 - 19 km ESE...|19 km ESE of Hawt...| 44|  1.7|     ml|2025-04-25 23:40:...|20

In [4]:
# Display the new dataframe with the country codes
display(df_with_location)

StatementMeta(, e2343822-04e0-4b3c-8cfc-e8502b405780, 6, Finished, Available, Finished)

NameError: name 'df_with_location' is not defined

In [ ]:
# Add a new column with significance classification
df_with_location_sig_class = \
                            df_with_location.\
                                withColumn('sig_class', 
                                            when(col("sig") < 100, "Low").\
                                            when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                            otherwise("High")
                                            )

In [ ]:
# Display the new table with significance classification

display(df_with_location_sig_class)

In [ ]:
# Append the data and save as the table for Gold Medallion

df_with_location_sig_class.write.mode('append').saveAsTable('earthquake_events_gold')